In [1]:
!nvidia-smi

import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="2"

import numpy as np
import cv2
import matplotlib.pyplot as plt
import SimpleITK as sitk
import cv2
from tqdm import tqdm

import sys
sys.path.append(os.path.abspath('/workspace/sunggu'))
sys.path.append(os.path.abspath('/workspace/sunggu/4.Dose_img2img'))
sys.path.append(os.path.abspath('/workspace/sunggu/MONAI'))
from sunggu_utils import check_value, take_list, plot_confusion_matrix, list_sort_nicely, find_dir, plot_3D

Thu May 27 02:45:27 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.73.01    Driver Version: 460.73.01    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  TITAN RTX           Off  | 00000000:02:00.0 Off |                  N/A |
| 42%   64C    P2    82W / 280W |   1618MiB / 24220MiB |     34%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  TITAN RTX           Off  | 00000000:03:00.0 Off |                  N/A |
| 42%   

# Setup imports

In [2]:
import glob
import torch
import os
import shutil
import tempfile
import matplotlib.pyplot as plt

import monai
from monai.utils import first, set_determinism
from monai.apps import download_and_extract
from monai.config import print_config
from monai.data import CacheDataset, DataLoader, Dataset
from monai.inferers import sliding_window_inference
from monai.losses import DiceLoss, GeneralizedDiceLoss, FocalLoss, TverskyLoss, NoiseRobustDiceLoss
from monai.metrics import compute_meandice, DiceMetric
from monai.transforms import (
    AddChanneld,
    Compose,
    CropForegroundd,
    LoadNiftid,
    Lambdad,
    Orientationd,
    RandCropByPosNegLabeld,
    ScaleIntensityRanged,
    Spacingd,
    ToTensord,
    RandScaleIntensityd,
    RandGaussianNoised,
    RandFlipd,
    RandZoomd,
    RandGaussianNoised,
    RandGaussianSmoothd,
    RandShiftIntensityd,
    SpatialPadd,
    RandAffined,
    CastToTyped,
    DeleteItemsd,
    FgBgToIndicesd,
    Rand3DElasticd,
    RandZoomd,
    Rand2DElasticd,
    RandWeightedCropd,
    AsDiscrete,
)

print_config()

MONAI version: 0+unknown
Python version: 3.6.9 (default, Oct  8 2020, 12:12:24)  [GCC 8.4.0]
OS version: Linux (4.4.0-210-generic)
Numpy version: 1.19.5
Pytorch version: 1.6.0
MONAI flags: HAS_EXT = False, USE_COMPILED = False

Optional dependencies:
Pytorch Ignite version: NOT INSTALLED or UNKNOWN VERSION.
Nibabel version: 3.2.1
scikit-image version: 0.17.2
Pillow version: 8.1.0
Tensorboard version: 2.4.1
gdown version: NOT INSTALLED or UNKNOWN VERSION.
TorchVision version: 0.7.0
ITK version: NOT INSTALLED or UNKNOWN VERSION.
tqdm version: 4.56.0

For details about installing the optional dependencies, please visit:
    https://docs.monai.io/en/latest/installation.html#installing-the-recommended-dependencies



In [3]:
import random

random_seed = 0
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed) # if use multi-GPU
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)
random.seed(random_seed)

set_determinism(seed=0)

## 파일 이름 바꾸기 (환자명 빼기)

In [ ]:
import os
for i in total_list:
    os.makedirs(os.path.dirname(i.replace("CHO_KYE_HWA_M80", "PATIENT_A")), exist_ok=True)
    shutil.copy(i, i.replace("CHO_KYE_HWA_M80", "PATIENT_A"))

# 3mm sinogram 폴더 정렬 및 이동

In [4]:
# total_list = list_sort_nicely(glob.glob("/workspace/sunggu/4.Dose_img2img/goto_sinogram/brain_3mm/*/*/Brain_Axial_B40f*/*"))
name_list = list_sort_nicely(glob.glob("/workspace/sunggu/4.Dose_img2img/goto_sinogram/brain_3mm/*"))

In [5]:
name = []
for i in name_list:
    name.append(i.split('/')[-1])

In [6]:
import shutil
import os 

for n in name:
    order_list = list_sort_nicely(glob.glob("/workspace/sunggu/4.Dose_img2img/goto_sinogram/brain_3mm/"+n+"/*"))
    
    noise_X = list_sort_nicely(glob.glob(order_list[0] + "/Brain_Axial_B40f*/*"))
    for file in noise_X:
#         print(file.replace("/"+n+"/", "/"+n+"/X/").replace('/goto_sinogram/brain_3mm/', '/dataset/[sinogram]Brain_3mm_DCM/'))
        move_to = file.replace("/"+n+"/", "/"+n+"/X/").replace('/goto_sinogram/brain_3mm/', '/dataset/[sinogram]Brain_3mm_DCM/')
        os.makedirs(os.path.dirname(move_to), mode=0o777, exist_ok=True)
        shutil.copy(file, move_to)
    
    noise_20 = list_sort_nicely(glob.glob(order_list[1] + "/Brain_Axial_B40f*/*"))
    for file in noise_20:
#         print(file.replace("/"+n+"/", "/"+n+"/20/").replace('/goto_sinogram/brain_3mm/', '/dataset/[sinogram]Brain_3mm_DCM/'))
        move_to = file.replace("/"+n+"/", "/"+n+"/20/").replace('/goto_sinogram/brain_3mm/', '/dataset/[sinogram]Brain_3mm_DCM/')
        os.makedirs(os.path.dirname(move_to), mode=0o777, exist_ok=True)
        shutil.copy(file, move_to)
    
    noise_40 = list_sort_nicely(glob.glob(order_list[2] + "/Brain_Axial_B40f*/*"))
    for file in noise_40:
#         print(file.replace("/"+n+"/", "/"+n+"/40/").replace('/goto_sinogram/brain_3mm/', '/dataset/[sinogram]Brain_3mm_DCM/'))
        move_to = file.replace("/"+n+"/", "/"+n+"/40/").replace('/goto_sinogram/brain_3mm/', '/dataset/[sinogram]Brain_3mm_DCM/')
        os.makedirs(os.path.dirname(move_to), mode=0o777, exist_ok=True)
        shutil.copy(file, move_to)
    
    noise_60 = list_sort_nicely(glob.glob(order_list[3] + "/Brain_Axial_B40f*/*"))
    for file in noise_60:
#         print(file.replace("/"+n+"/", "/"+n+"/60/").replace('/goto_sinogram/brain_3mm/', '/dataset/[sinogram]Brain_3mm_DCM/'))        
        move_to = file.replace("/"+n+"/", "/"+n+"/60/").replace('/goto_sinogram/brain_3mm/', '/dataset/[sinogram]Brain_3mm_DCM/')
        os.makedirs(os.path.dirname(move_to), mode=0o777, exist_ok=True)
        shutil.copy(file, move_to)        
    
    noise_80 = list_sort_nicely(glob.glob(order_list[4] + "/Brain_Axial_B40f*/*"))
    for file in noise_80:
#         print(file.replace("/"+n+"/", "/"+n+"/80/").replace('/goto_sinogram/brain_3mm/', '/dataset/[sinogram]Brain_3mm_DCM/'))
        move_to = file.replace("/"+n+"/", "/"+n+"/80/").replace('/goto_sinogram/brain_3mm/', '/dataset/[sinogram]Brain_3mm_DCM/')
        os.makedirs(os.path.dirname(move_to), mode=0o777, exist_ok=True)
        shutil.copy(file, move_to)   

# 5mm sinogram 폴더 정렬 및 이동

In [20]:
total_list = list_sort_nicely(glob.glob("/workspace/sunggu/4.Dose_img2img/goto_sinogram/brain_5mm/*/*/*/Brain_Axial_B40f*/*"))

In [22]:
for file in total_list:
    move_to = file.replace('/goto_sinogram/brain_5mm/', '/dataset/[sinogram]Brain_5mm_DCM/')
    os.makedirs(os.path.dirname(move_to), mode=0o777, exist_ok=True)
    
    shutil.copy(file, move_to)   